# 記得常常用markdown 來分隔不同部分的程式碼

## 以下是應該考慮的Feature!! 如果已完成 就用#標記掉

* 一個購買行為發生之前的features:

In [ ]:
# 多少個date

In [ ]:
# 多少個session

In [ ]:
# 多少個session series
名稱名 變形date 
變數名session_series_list

# 把time 塞進session list之後計算出所有東西，平均頁面瀏覽時間，session初始時間，把time 塞進session list之後計算出所有東西

In [ ]:
# 一個 session 內的頭尾時間差距(session使用期間)，"沒有尾的(比如一個session只有一個瀏覽行為)，那不理她。兩筆資料以上計算(除上n-1)"
名稱名 一個session平均頁面瀏覽時間
變數名 #session_viewtime_list
        session_avg_viewtime_list

In [ ]:
#一個 session series 內的所有"session使用期間"相加
名稱名　一個session series平均頁面瀏覽時間
變數名 #session_series_viewtime_list
        session_series_avg_viewtime_list

In [ ]:
#一個 session 內"第一個值" 的廣告來源以及瀏覽來源(web, app)，"需要另外寫購買前30天內的廣告來源分布"
#一個 session 內的頭初始時間(用來計算時間加權廣告貢獻)，"也可用來計算使用者活躍時間"(看是在白天,晚上,星期幾瀏覽跟購物)

名稱名  一個session的第一筆資料
變數名  session_first_data_list


In [ ]:
#一個 購買週期內的所有瀏覽商品 

In [ ]:
#一個 購買週期內的所有加入購物車的商品 "注意，有這些特徵值就可以計算購物車轉換率，不必另外寫函數。"

In [ ]:
#一個 購買週期內的所有加入Fav的商品

In [ ]:
#一個 購買週期內的所有購買商品

In [ ]:
# 一個 購買週期內整體轉換率 => 多少個session 會變成購買行為(線下或線上)

In [ ]:
一個 購買週期內最後產生的購買的那些商品頁面，在接下來7天是否有瀏覽，"為了看是否該使用者會去貢獻comments"

In [ ]:
#計算 該次購物的總金額，折扣金額，購物的時間

In [ ]:
# 計算以月為單位的freqency 以及 購物金額

In [ ]:
# 計算view, cart fav 轉換率

## 還要思考的點

* 線上購買行為發生後，如果繼續瀏覽網頁，購買後的行為應歸屬何處? Session 或 Session series 要重算嗎?

# 先用open 方法打開，好處是一行一行讀進來可以知道每一個數字的意義

In [3]:
import math
file_handler = open('45826522P9.txt', 'r')

## 閱讀方法是對open這個物件，使用readlines()

In [4]:
lines = file_handler.readlines()

## 經過split後給予變數名來做事情才知道在寫啥

In [5]:
line = lines[0]
time, ad, session, behavior, source, number = line.split(',')

In [6]:
print(time, ad, session, behavior, source, number)
print(behavior == 'Purchase')

2018-03-06 00:23:22 GoogleCpc 1 ViewSalePageCategory WEB 55165

False


# 接著一行一行讀進來做事情，首要處理是session的連續問題，那我們先看看有幾個date，然後有幾個連續的session(比date的數量還少才正確)

In [8]:
from datetime import datetime, timedelta

## 使用 in list的方法來讀取每一行

## 此為控制是否有換天 換session 等函數 返回值為True or False

In [10]:
Change_day_Flag = False #代表是否有曾經換過日期

def change_date(previous_time, time):
    global Change_day_Flag #控制 Flag 用
    
    if time.date() != previous_time.date():
        Change_day_Flag = True
        return True
    else:
        return False
    
def change_session(previous_session, session, previous_time, time, previous_source, source):
    if ( time - previous_time >  timedelta(seconds = 1800) )  or source != previous_source : 
        return True
    elif  time.date() == previous_time.date() and session != previous_session :
        return True
    else:
        return False

def change_session_series(previous_session, session, previous_time, time, previous_source, source):
    global Change_day_Flag #控制 Flag 用
    
    if Change_day_Flag == True:
        if change_session(previous_session, session, previous_time, time, previous_source, source):
            Change_day_Flag = False
            return True
    else:
        return False

## 開始計算與session有關的東西

In [13]:
line = lines[0] #第一row

time, ad, session, behavior, source, number = line.split(',')

# 紀錄連續的session跟time
previous_time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S') #上一筆的時間
previous_session = session #上一筆的session
previous_source = source #上一筆的source

# 讀之後的資料!
Change_day_Flag = False #代表是否有曾經換過日期

# 把date記下來，以及把session記下來，session counter記下來
date_list = [previous_time.date()]
session_series_list = []
session_list = []

#GISH: 各SESSION第一筆資料
session_first_data_list =[line.split(',')]

# 暫時的sessionseries and session holder
tem_session_list = [previous_time]
tem_session_series_list = [previous_time]

for line in lines[1:]: #從第二row 開始
    time, ad, session, behavior, source, number = line.split(',')
    time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    
    # 日期轉換
    if change_date(previous_time, time): #如果換天
        date_list.append(time.date())
    
    # session轉換
    if change_session(previous_session, session, previous_time, time, previous_source, source) :#如果換session
        session_list.append(tem_session_list) #GISH
        tem_session_list = [time] #GISH
        session_first_data_list.append(line.split(',')) #G 
    else: 
        tem_session_list.append(time) #G
    # session series轉換    
    if change_session_series(previous_session, session, previous_time, time, previous_source, source):
        session_series_list.append(tem_session_series_list)
        tem_session_series_list = [time]   
    else:
        tem_session_series_list.append(time)
        
    #記得更新上一本的資料
    previous_time = time #上一筆的time
    previous_session = session #上一筆的session
    previous_source = source #上一筆的source
    
# 最後一筆資料記得添加
session_series_list.append(tem_session_series_list)
session_list.append(tem_session_list)


In [17]:
session_first_data_list

[['2018-03-08 23:56:39', 'GoogleCpc', '8', 'ViewSalePage', 'WEB', '3932309'],
 ['2018-03-09 00:02:11',
  'GoogleCpc',
  '2',
  'ViewSalePageCategory',
  'WEB',
  '55165'],
 ['2018-03-10 00:38:26', 'Others', '1', 'ViewSalePage', 'WEB', '4106259'],
 ['2018-03-10 00:42:07',
  'Others',
  '2',
  'ViewSalePageCategory',
  'WEB',
  '35222']]

# GISH 算出每個SESSION的瀏覽時間

In [28]:
session_list

[[datetime.datetime(2018, 3, 8, 23, 56, 39),
  datetime.datetime(2018, 3, 8, 23, 57, 32),
  datetime.datetime(2018, 3, 8, 23, 58, 38),
  datetime.datetime(2018, 3, 8, 23, 59, 9),
  datetime.datetime(2018, 3, 9, 0, 0, 14),
  datetime.datetime(2018, 3, 9, 0, 0, 40),
  datetime.datetime(2018, 3, 9, 0, 2, 6)],
 [datetime.datetime(2018, 3, 9, 0, 2, 11),
  datetime.datetime(2018, 3, 9, 0, 2, 24),
  datetime.datetime(2018, 3, 9, 0, 2, 40),
  datetime.datetime(2018, 3, 9, 0, 3, 27),
  datetime.datetime(2018, 3, 9, 0, 3, 44),
  datetime.datetime(2018, 3, 9, 0, 3, 52)],
 [datetime.datetime(2018, 3, 10, 0, 38, 26)],
 [datetime.datetime(2018, 3, 10, 0, 42, 7)]]

In [31]:
#session_viewtime_list = [] # 此為儲存每個session的瀏覽時間，如果只有一筆資料就為0
session_avg_viewtime_list = []
for session_times in session_list:
    session_viewtime_list.append(session_times[-1]-session_times[0])
    try:
        session_avg_viewtime_list.append((session_times[-1]-session_times[0])/(len(session_times)-1))
    except:
        session_avg_viewtime_list.append(timedelta(0))

In [30]:
#session_viewtime_list

[datetime.timedelta(seconds=327),
 datetime.timedelta(seconds=101),
 datetime.timedelta(0),
 datetime.timedelta(0)]

In [32]:
session_avg_viewtime_list

[datetime.timedelta(seconds=54, microseconds=500000),
 datetime.timedelta(seconds=20, microseconds=200000),
 datetime.timedelta(0),
 datetime.timedelta(0)]

# GISH 算出每個SESSION SERIES瀏覽時間

In [38]:
#session_series_viewtime_list = [] # 此為儲存每個session series的瀏覽時間，如果只有一筆資料就為0
session_series_avg_viewtime_list = []
for session_series_times in session_series_list:
    viewtime = session_series_times[-1]-session_series_times[0]
    session_series_viewtime_list.append(viewtime)
    try:
        session_series_avg_viewtime_list.append(viewtime/(len(session_series_times)-1))
    except:
        session_series_avg_viewtime_list.append(timedelta(0))
        

In [37]:
#session_series_viewtime_list

[datetime.timedelta(seconds=327),
 datetime.timedelta(seconds=101),
 datetime.timedelta(seconds=221)]

In [39]:
session_series_avg_viewtime_list

[datetime.timedelta(seconds=54, microseconds=500000),
 datetime.timedelta(seconds=20, microseconds=200000),
 datetime.timedelta(seconds=221)]

# 告一段落

In [55]:
date_list,session_list,session_series_list

([datetime.date(2018, 3, 8),
  datetime.date(2018, 3, 9),
  datetime.date(2018, 3, 10)],
 [['8', '8', '8', '8', '1', '1', '1'],
  ['2', '2', '2', '2', '2', '2'],
  ['1'],
  ['2']],
 [['8', '8', '8', '8', '1', '1', '1'],
  ['2', '2', '2', '2', '2', '2'],
  ['1', '2']])

## 我弄了一個test的資料
## 照理說 test的第0~5筆 將會有兩個date_count, 但只有一個session，一個session series
## 不過如果是全部資料 將會有3個date, 4個session, 3個session seires(修改的date定義)

In [5]:
test = [

"2018-03-08 23:56:39,GoogleCpc,8,ViewSalePage,WEB,3932309",
"2018-03-08 23:57:32,GoogleCpc,8,ViewSalePageCategory,WEB,51876", 
"2018-03-08 23:58:38,GoogleCpc,8,ViewSalePage,WEB,3991800",
"2018-03-08 23:59:09,GoogleCpc,8,ViewSalePageCategory,WEB,51876",
"2018-03-09 00:00:14,Direct,1,ViewSalePageCategory,WEB,51876",
"2018-03-09 00:00:40,Direct,1,ViewSalePageCategory,WEB,51876",
"2018-03-09 00:02:06,Direct,1,ViewSalePageCategory,WEB,51876", 
"2018-03-09 00:02:11,GoogleCpc,2,ViewSalePageCategory,WEB,55165",
"2018-03-09 00:02:24,GoogleCpc,2,ViewSalePageCategory,WEB,182295",
"2018-03-09 00:02:40,GoogleCpc,2,ViewSalePage,WEB,4161481",
"2018-03-09 00:03:27,GoogleCpc,2,ViewSalePageCategory,WEB,55165",
"2018-03-09 00:03:44,GoogleCpc,2,ViewSalePageCategory,WEB,182295",
"2018-03-09 00:03:52,GoogleCpc,2,ViewSalePage,WEB,4161481",
"2018-03-10 00:38:26,Others,1,ViewSalePage,WEB,4106259",
"2018-03-10 00:42:07,Others,2,ViewSalePageCategory,WEB,35222"
]

In [6]:
lines = test

In [29]:
line = lines[0] #第一row
time, ad, session, behavior, source, number = line.split(',')

# 紀錄連續的session跟time
previous_time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S') #上一筆的時間
previous_session = session #上一筆的session
previous_source = source #上一筆的source

# 讀之後的資料!
Change_day_Flag = False #代表是否有曾經換過日期
date_counter = 1 # 以時間來計算，11231 會有兩天
session_series_counter = 1 #以序列來計算，11231只有一個序列
session_counter = 1 #以session來計算，11231會有3個session，分別是 [11, 2, 31]

# 把date記下來，以及把session記下來，session counter記下來
date_list = [previous_time.date()]
session_series_list = []
session_list = []


# 暫時的sessionseries and session holder
tem_session_list = [session]
tem_session_series_list = [session]

for line in lines[1:]: #從第二row 開始
    time, ad, session, behavior, source, number = line.split(',')
    time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    

    
    
    # 日期轉換
    if change_date(previous_time, time): #如果換天
        date_counter += 1
        date_list.append(time.date())
    
    # session轉換
    if change_session(previous_session, session, previous_time, time, previous_source, source) :#如果換session
        session_counter += 1
        session_list.append(tem_session_list)
        tem_session_list = [session]   
    else:
        tem_session_list.append(session)
        
        
    # session series轉換    
    if change_session_series(previous_session, session, previous_time, time, previous_source, source):
        session_series_counter += 1
        session_series_list.append(tem_session_series_list)
        tem_session_series_list = [session]   
    else:
        tem_session_series_list.append(session)
        
    #記得更新上一本的資料
    previous_time = time #上一筆的time
    previous_session = session #上一筆的session
    previous_source = source #上一筆的source
    
# 最後一筆資料記得添加
session_series_list.append(tem_session_series_list)
session_list.append(tem_session_list)


In [30]:
date_counter, session_counter , session_series_counter

(3, 4, 3)

In [31]:
date_list

[datetime.date(2018, 3, 8),
 datetime.date(2018, 3, 9),
 datetime.date(2018, 3, 10)]

In [32]:
session_list

[['8', '8', '8', '8', '1', '1', '1'],
 ['2', '2', '2', '2', '2', '2'],
 ['1'],
 ['2']]

In [33]:
session_series_list

[['8', '8', '8', '8', '1', '1', '1'],
 ['2', '2', '2', '2', '2', '2'],
 ['1', '2']]

# ....用這種方式寫程式，別改那9個變數名
## time, ad, session, behavior, source, number, previous_time, previous_session, previous_source

## keep salepage items

In [126]:
def is_buying(behavior): #看是不是購買行為
    if behavior in ['Purchase', 'Offline']:
        return True
    else:
        return False
    
def is_salepage_or_cart_or_fav(behavior): ##看是不是salepage 或者 購物車 或者 加入最愛行為
    if behavior in ['ViewSalePage', 'Cart', 'Fav']:
        return True
    else:
        return False
    
    
def update_product_dictionary(product_id, behavior, time = ''): 
    #把瀏覽的商品ID給記錄下來，每個id對應5個元素，[view, cart, fav, buy, (last_time, last_type)]
    global product_dictionary
    
    if product_id not in product_dictionary:
        product_dictionary[product_id] = [0,0,0,0,('', '')]

    if behavior in ['ViewSalePage', 'Cart', 'Fav']:
        for i in range(3):
            if behavior == ['ViewSalePage', 'Cart', 'Fav'][i]:
                product_dictionary[product_id][i] +=1
                break
        product_dictionary[product_id][4] = (time, behavior)
        
    elif behavior in ['Purchase', 'Offline']:
        product_dictionary[product_id][3] +=1
            

    
def update_purchase_dictionary(product_id, Status):
    global purchase_dictionary
    if Status == 'Fin':
        if product_id not in purchase_dictionary['finish_id'] :
            purchase_dictionary['finish_id'].append(product_id)
    

    elif Status == 'Ret':
        if product_id not in purchase_dictionary['return_id'] :
            purchase_dictionary['return_id'].append(product_id)
        
def update_purchase_time(time, amount, discount, where): #第一筆購買資料出現後 要更新該筆購物的時間地點金額
    global purchase_dictionary
    

    purchase_dictionary['amount'] = amount
    purchase_dictionary['time'] = time
    purchase_dictionary['discount'] = discount
    purchase_dictionary['where'] = 'Online' if where=='Purchase' else 'Offline'

def is_first_purchase_row(): #看該筆購買行為是不是一個購買行為的第一次出現購買紀錄
    global Next_purchase_Flag
    if Next_purchase_Flag == False: #將會跟新一個flag，用來知道是否要更新
        Next_purchase_Flag = True
        return True
    else:
        return False
    
def is_same_purchase(time):  #看該筆購買行為是不是同一個單號的東西
    if  time == purchase_dictionary['time']:
        return True
    else:
        return False
    
def needs_save_everything(time = '', behavior = ''):#看是不是需要更新
    global Next_purchase_Flag, tem_buying_behaviour, purchase_dictionary
    

    
    if Next_purchase_Flag == True :#如果有之前有購買紀錄出現
    
        if time != purchase_dictionary['time'] or behavior not in ['Purchase', 'Offline']: #看看是否是連續的購買行為
            
            #如果是的話，看看這個購買行為有沒有意義(是不是有status是finish)
            is_valid_purchase = False
            for a_id,a_status in tem_buying_behaviour:
                if a_status =='Fin':
                    is_valid_purchase = True
                    break
            
            
            
            if is_valid_purchase: #如果是有效的購買行為 將會return True 並且之後會呼叫save_everything()
                Next_purchase_Flag = False
                return True
            
            else: #不然的話就把 tem_buying_behaviour，Next_purchase_Flag 給歸零
                tem_buying_behaviour = []
                Next_purchase_Flag = False
                return False

    return False
    
def save_everything(): #更新全部的東西
    global purchase_dict_records, product_dictionary_records, purchase_dictionary, product_dictionary, tem_buying_behaviour
    
    
    for a_id,a_status in tem_buying_behaviour:
        update_purchase_dictionary(a_id,a_status)
        update_product_dictionary(a_id, 'Purchase')
    
    purchase_dict_records.append(purchase_dictionary)
    product_dictionary_records.append(product_dictionary)
    
    # reset those value
    purchase_dictionary = dict() 
    purchase_dictionary['finish_id'] = []
    purchase_dictionary['return_id'] = []
    product_dictionary = dict()
    tem_buying_behaviour = [] 
    
    

In [127]:
# 紀錄product dictionary
purchase_dict_records = [] #放dict, purchase_dictionary
product_dictionary_records = [] #放dict，紀錄product_dict

purchase_dictionary = dict() #紀錄每個purchase的狀態 ，{amount:總金額, discount:折扣金額, finish_id:[], return_id:[], time:時間, where:地點}
purchase_dictionary['finish_id'] = []
purchase_dictionary['return_id'] = []


product_dictionary = dict() #紀錄每個product_id的狀態，{ id : (看次數、購物車次數、FAV次數、買次數, (最後一次使用的時間與type)) } 


tem_buying_behaviour = [] #紀錄遇到purchase後 的Status以及productID狀態，為了避免concel or fail or 只有退貨的狀況


In [128]:
Next_purchase_Flag = False

for line in lines: #從第一row 開始

    time, ad, session, behavior, source, number = line.strip('\n').split(',')
    time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')

    
    
    if needs_save_everything(time, behavior): #看需不需要儲存購買行為了!
        save_everything()#存東西
        
    if is_salepage_or_cart_or_fav(behavior): #看是不是salepage 或者 購物車 或者 加入最愛行為
        update_product_dictionary(number, behavior, time) #把瀏覽的商品ID給記錄下來
            
        
        
        
    elif is_buying(behavior): #看是不是購買行為
        
        # try, except 是為了處理有些線下行為不知道對應的線上商品ID
        try:
            product_id, amount, discount, Status = number.split()
        except:
            amount, discount, Status = number.split()
            product_id = ''
        
        if is_first_purchase_row(): #看該筆購買行為是不是一個購買行為的第一次出現購買紀錄
            
            update_purchase_time(time, amount, discount, behavior)  #第一筆才要記錄
            tem_buying_behaviour.append( (product_id, Status) )
        
        
        elif is_same_purchase(time):
            tem_buying_behaviour.append( (product_id, Status) )
      




    
    
    
        
    #記得更新上一本的資料
    previous_time = time #上一筆的time
    previous_session = session #上一筆的session
    previous_source = source #上一筆的source

# 怕最後一筆資料是購買資料:
if needs_save_everything():
    save_everything()

In [129]:
len(purchase_dict_records), len(product_dictionary_records)

(40, 40)

In [47]:
#purchase_dict_records[0], product_dictionary_records[0]

# TEST

In [40]:
test = [
"2018-03-22 13:28:14,,,Offline,,3160278 2660 -2000 Fin",
"2018-03-22 13:28:14,,,Offline,,5091125 2660 -2000 Fin",
"2019-02-26 07:34:42,GoogleCpc,3,ViewSalePageCategory,WEB,187928",
"2019-02-26 07:34:55,GoogleCpc,3,ViewSalePageCategory,WEB,51876",
"2019-02-26 07:35:48,GoogleCpc,3,ViewSalePage,WEB,5079977",
"2019-02-26 07:36:28,GoogleCpc,3,ViewSalePageCategory,WEB,51876",
"2019-02-26 07:36:32,GoogleCpc,3,ViewSalePageCategory,WEB,187928",
"2019-02-26 07:36:44,GoogleCpc,3,ViewSalePageCategory,WEB,182295",
"2019-02-26 07:37:04,GoogleCpc,3,ViewSalePage,WEB,5199774",
"2019-02-26 07:37:47,GoogleCpc,3,Cart,WEB,5199774",
"2019-02-26 07:41:28,Direct,1,Purchase,WEB,5149422 3250 0 Fin",
"2019-02-26 07:41:28,Direct,1,Purchase,WEB,5164910 3250 0 Fin",
"2019-02-26 07:41:28,Direct,1,Purchase,WEB,5197781 3250 0 Fin",
"2019-02-26 08:26:31,GoogleCpc,4,ViewSalePageCategory,WEB,182295",
"2019-02-26 08:26:42,GoogleCpc,4,ViewSalePage,WEB,5199802",
"2019-02-26 08:26:47,GoogleCpc,4,Cart,WEB,5199802",
"2019-02-26 08:26:48,GoogleCpc,4,Purchase,WEB,5199802 9527 -487 Can",
"2019-02-26 08:26:52,GoogleCpc,4,ViewSalePageCategory,WEB,182295",
"2019-02-26 08:27:01,GoogleCpc,4,ViewSalePage,WEB,5199774",
"2019-02-26 08:31:27,GoogleCpc,4,ViewSalePageCategory,WEB,182295",
"2019-02-26 08:31:37,GoogleCpc,4,ViewSalePage,WEB,5199841",
"2019-02-26 08:31:45,GoogleCpc,4,ViewSalePageCategory,WEB,182295",
"2019-02-26 08:31:52,GoogleCpc,4,ViewSalePage,WEB,5199841",
"2019-02-26 08:58:30,GoogleCpc,4,ViewSalePageCategory,WEB,187928",
"2019-02-26 09:31:14,GoogleCpc,5,ViewSalePageCategory,WEB,86270",
"2019-02-26 09:37:16,Direct,3,Purchase,WEB,5199774 2860 0 Fin",
"2019-02-26 09:37:16,Direct,3,Purchase,WEB,5199802 2860 0 Fin",
"2019-02-26 09:37:47,GoogleCpc,5,ViewSalePageCategory,WEB,86270",
"2019-02-26 14:05:48,GoogleCpc,6,ViewSalePageCategory,WEB,86270",
"2019-03-22 13:28:14,,,Offline,,3160278 2660 -2000 Fin",
"2019-03-22 13:28:14,,,Offline,,5091125 2660 -2000 Fin"
]

In [27]:
Next_purchase_Flag = False

for line in test[0:]: #從第一row 開始

    time, ad, session, behavior, source, number = line.strip('\n').split(',')
    time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')

    
    
    if needs_save_everything(time, behavior): #看需不需要儲存購買行為了!
        save_everything()#存東西
        
    if is_salepage_or_cart_or_fav(behavior): #看是不是salepage 或者 購物車 或者 加入最愛行為
        update_product_dictionary(number, behavior, time) #把瀏覽的商品ID給記錄下來
            
        
        
        
    elif is_buying(behavior): #看是不是購買行為
        
        # try, except 是為了處理有些線下行為不知道對應的線上商品ID
        try:
            product_id, amount, discount, Status = number.split()
        except:
            amount, discount, Status = number.split()
            product_id = ''
        
        if is_first_purchase_row(): #看該筆購買行為是不是一個購買行為的第一次出現購買紀錄
            
            update_purchase_time(time, amount, discount, behavior)  #第一筆才要記錄
            tem_buying_behaviour.append( (product_id, Status) )
        
        
        elif is_same_purchase(time):
            tem_buying_behaviour.append( (product_id, Status) )
      




    
    
    
        
    #記得更新上一本的資料
    previous_time = time #上一筆的time
    previous_session = session #上一筆的session
    previous_source = source #上一筆的source

# 怕最後一筆資料是購買資料:
if needs_save_everything():
    save_everything()

In [32]:
len(purchase_dict_records), len(product_dictionary_records)

(4, 4)

In [28]:
purchase_dict_records[0], product_dictionary_records[0]

({'finish_id': ['3160278', '5091125'],
  'return_id': [],
  'amount': '2660',
  'time': datetime.datetime(2018, 3, 22, 13, 28, 14),
  'discount': '-2000',
  'where': 'Offline'},
 {'3160278': [0, 0, 0, 1, ('', '')], '5091125': [0, 0, 0, 1, ('', '')]})

In [29]:
purchase_dict_records[1], product_dictionary_records[1]

({'finish_id': ['5149422', '5164910', '5197781'],
  'return_id': [],
  'amount': '3250',
  'time': datetime.datetime(2019, 2, 26, 7, 41, 28),
  'discount': '0',
  'where': 'Online'},
 {'5079977': [1,
   0,
   0,
   0,
   (datetime.datetime(2019, 2, 26, 7, 35, 48), 'ViewSalePage')],
  '5199774': [1, 1, 0, 0, (datetime.datetime(2019, 2, 26, 7, 37, 47), 'Cart')],
  '5149422': [0, 0, 0, 1, ('', '')],
  '5164910': [0, 0, 0, 1, ('', '')],
  '5197781': [0, 0, 0, 1, ('', '')]})

In [30]:
purchase_dict_records[2], product_dictionary_records[2]

({'finish_id': ['5199774', '5199802'],
  'return_id': [],
  'amount': '2860',
  'time': datetime.datetime(2019, 2, 26, 9, 37, 16),
  'discount': '0',
  'where': 'Online'},
 {'5199802': [1, 1, 0, 1, (datetime.datetime(2019, 2, 26, 8, 26, 47), 'Cart')],
  '5199774': [1,
   0,
   0,
   1,
   (datetime.datetime(2019, 2, 26, 8, 27, 1), 'ViewSalePage')],
  '5199841': [2,
   0,
   0,
   0,
   (datetime.datetime(2019, 2, 26, 8, 31, 52), 'ViewSalePage')]})

In [31]:
purchase_dict_records[3], product_dictionary_records[3]

({'finish_id': ['3160278', '5091125'],
  'return_id': [],
  'amount': '2660',
  'time': datetime.datetime(2019, 3, 22, 13, 28, 14),
  'discount': '-2000',
  'where': 'Offline'},
 {'3160278': [0, 0, 0, 1, ('', '')], '5091125': [0, 0, 0, 1, ('', '')]})

# 計算freqency 以及 金額大小，首先必須要有purchase_dict_records

In [130]:
def update_F_M(purchase_time, purchase_amount):
    global F_list, M_list
    
    
    if purchase_time.year == 2018:
        i = purchase_time.month - 3
    elif purchase_time.year == 2019:
        i = purchase_time.month + 9
    
    F_list[i] +=1
    M_list[i] += purchase_amount

In [131]:
F_list = [0]*14
M_list = [0]*14
for a_record in purchase_dict_records:
    purchase_time = a_record['time']
    purchase_amount = int( a_record['amount'] )
    update_F_M(purchase_time, purchase_amount)
    

In [134]:
for i in range(14):
    print( F_list[i], M_list[i])

0 0
1 2560
4 21200
4 10040
7 36500
0 0
1 1480
1 2460
0 0
2 3660
7 39420
7 21770
4 12740
2 8090


# 算 點擊、購物車、FAV 轉換率

In [135]:
def update_conversion_list(view, cart, fav, product_id):
    global conversion_list
    
    if view !=0:
        conversion_list[0].add(product_id)
    if cart !=0:
        conversion_list[1].add(product_id)
    if fav !=0:
        conversion_list[2].add(product_id)
        
def update_conversion_metrics(conversion_list,  purchase_dict):
    global conversion_metrics_all
    
    def conversion_rate(buy_set, other_set):
        if len(other_set) == 0:
            return None
        else:
            return round( len(  buy_set.intersection(other_set)   ) /  len(other_set) , 2)
    buy_set = set( purchase_dict['finish_id'] )
    view_set, cart_set, fav_set = conversion_list
    
    
    v_c = conversion_rate(buy_set, view_set)
    c_c = conversion_rate(buy_set, cart_set)
    f_c = conversion_rate(buy_set, fav_set)
    
    conversion_metrics_all.append(  [v_c, c_c, f_c ])
    
    

In [136]:
'定義是 view_conversion_rate = 所有有被看過的商品，其最後會被買的比率' ,
'定義是 cart_conversion_rate = 所有有被cart的商品，其最後會被買的比率' ,
'定義是  fav_conversion_rate = 所有有被 fav的商品，其最後會被買的比率' 
conversion_metrics_all = [] #裡面放每一次交易的轉換率list = [view 轉換率, cart 轉換率, fav轉換率]

In [137]:
for i in range(len(product_dictionary_records)):
    a_dict = product_dictionary_records[i]
    conversion_list = [set(), set(), set()]
    
    cnt = 0
    for product_id in a_dict.keys():
        view, cart, fav, buy, (time, last) = a_dict[product_id]
        update_conversion_list(view, cart, fav, product_id)
        
    update_conversion_metrics(conversion_list, purchase_dict_records[i])

* total conversion rate ( 買次數/ session次數)

In [138]:
len(purchase_dict_records) / len(session_list)

0.033984706881903144

* [view 轉換率, cart 轉換率, fav轉換率]

In [139]:
for i in range(len(conversion_metrics_all)):
    print( purchase_dict_records[i]['where'], purchase_dict_records[i]['time'], '\n',conversion_metrics_all[i], purchase_dict_records[i]['amount'])

Online 2018-04-19 22:39:42 
 [0.01, 0.25, 0.0] 2560
Offline 2018-05-01 19:04:15 
 [0.16, 1.0, 0.5] 16080
Offline 2018-05-14 15:15:05 
 [0.02, 0.0, None] 90
Online 2018-05-30 07:54:16 
 [0.06, 1.0, 1.0] 3450
Online 2018-05-31 20:24:28 
 [0.2, 1.0, None] 1580
Online 2018-06-15 00:27:06 
 [0.04, 1.0, 0.0] 2560
Online 2018-06-22 20:32:32 
 [0.02, 0.33, None] 2360
Online 2018-06-26 01:09:24 
 [0.12, 1.0, None] 2760
Offline 2018-06-27 21:20:23 
 [0.0, None, None] 2360
Offline 2018-07-08 17:41:10 
 [0.02, 0.0, None] 1280
Online 2018-07-10 01:03:23 
 [0.07, 0.5, None] 4040
Online 2018-07-11 00:14:08 
 [0.13, 0.8, None] 4530
Offline 2018-07-21 17:20:13 
 [0.02, 0.0, None] 18380
Online 2018-07-24 07:59:06 
 [0.03, 0.33, None] 3340
Online 2018-07-25 18:57:22 
 [0.06, 1.0, 0.0] 1480
Online 2018-07-31 01:11:27 
 [0.03, None, None] 3450
Online 2018-09-28 00:01:13 
 [0.01, 0.5, 0.0] 1480
Online 2018-10-09 07:07:43 
 [0.07, 1.0, None] 2460
Online 2018-12-14 12:57:39 
 [0.01, 0.0, None] 2180
Online 201

# 原本Gish的東西

In [ ]:
import pandas as pd
import math
df = pd.read_csv('45826522P9.txt',header = None) 
"建議變數名寫df ，這是dataframe object 而不是file object"

In [170]:
#回傳一個陣列，裡面有  除了第一列(也就是時間列)的所有value count
#呼叫時parameters放入要讀取的檔案
def countValues(file):
    file_header = list(file)
    unique_count = {}
    for i in range(1,len(file_header)):  #刻意不算時間欄位
        valueCounts = file[file_header[i]].value_counts()
        for j in range(len(valueCounts.index)):
            unique_count[valueCounts.index[j]] = valueCounts.values[j]
    return unique_count

#以下沒用到，以後要用可以參考
#unique_names = file[file_header[i]].value_counts().index.tolist() #算出個欄位unique值
#unique_count[valueCounts.indexs] = valueCounts.values
        

In [171]:
countValuesList = countValues(file)

In [172]:
def activityCount(file):  #使用這個function請先確定資料是照日期排序，且第二欄位為TrafficSource
    sessionJumpCount = 0 #算session跳另一組的次數
    rowCount = 0
    for i in range(len(file.index)):
        if i==0: 
            rowCount =1 
            continue
        if file[2][i] == file[2][i-1]: #如果session連續
            rowCount += 1
        else:
            #print("換另一session")
            sessionJumpCount += 1
            rowCount +=1
            
    print("rowCount = " + str(rowCount))
    print("sessionJumpCount =" +str(sessionJumpCount))
    # print( math.log2(rowCount) + sessionJumpCount) #97
    return math.sqrt(rowCount)+sessionJumpCount

In [173]:
activityCount(file)

rowCount = 10173
sessionJumpCount =1045


1145.8612908900138

In [174]:
def purchaseActivityCount(countValuesList): #執行過countValues函數，把那個函數的return值丟進來才能算
    purchaseActivityDict = {}
    favCount = countValuesList['Fav']
    cartCount = countValuesList['Cart']
    purchaseCount = countValuesList['Purchase']
    purchaseCartConversionRate = purchaseCount/cartCount # 購物次數/cart次數
    favPurchaseConversionRate = favCount/purchaseCount # fav次數/購物次數
    #所以purchaseCartConversionRate*favCartConversionRate =  fav次數/cart次數
    favCartConversionRate = purchaseCartConversionRate*favPurchaseConversionRate # fav/Cart
    purchaseActivityDict.update({'favCount' : favCount,
                                'cartCount' : cartCount,
                                'purchaseCount' : purchaseCount,
                                'purchaseCartConversionRate' : purchaseCartConversionRate,
                                'favPurchaseConversionRate' : favPurchaseConversionRate,
                                'favCartConversionRate' : favCartConversionRate})
    return purchaseActivityDict

In [175]:
purchaseActivityCount(countValuesList)

{'favCount': 37,
 'cartCount': 142,
 'purchaseCount': 64,
 'purchaseCartConversionRate': 0.4507042253521127,
 'favPurchaseConversionRate': 0.578125,
 'favCartConversionRate': 0.2605633802816901}

In [12]:
# print(file[3][1])

ViewSalePage


In [ ]:
# for i, row in enumerate(file.values):
#     print(i)
#     print(type(row[0]))
#     print(file.values[i-1])

In [ ]:
# for i, row in enumerate(file.values):
#     #print((pd.to_datetime(row[0]) - pd.to_datetime(file.values[i-1][0])).total_seconds())
#     print(i)
#     if(i%3==0):
#         i+=2

In [374]:
# print(len(file.index))

49


In [429]:
# txt = "8888 55 44"
# txt.split(" ")
#print(file[0][0].split(" ")[0])

Timestamp('2018-04-08 19:36:45')

# 開始做事情?

### a_purchase_segment_row : 什麼?

In [527]:
import datetime
def purchaseBeforeCount(file):
    
    one_empty_row = pd.DataFrame([0])
    file = file.append(one_empty_row,ignore_index=True)
    
    #print(file)
    
    purchase_segment_dict = {}
    a_purchase_segment_row = 0
    a_purchase_segment_product_list = {}
    a_purchase_segment_source_type = {}
    a_purchase_segment_SessionCount=0   #　session的跳動count
    a_purchase_segment_SessionSeriesCount = 1 # session序列跳下一個session序列的跳動count
#     a_purchase_segment_SessionCount_totaltime = 0 # 每一個session 在一個purchase segment下的 總瀏覽時間
#     a_purchase_segment_SessionSeriesCount_totaltime = 0 # 每一個sessionSeries 在一個purchase segment下的 總瀏覽時間
    
    a_purchase_segment_behavior_type ={}
    a_purchase_segment_traffic_source ={}
    a_purchase_order = {} #訂單的東西
    purchase_orders = []
    purchase_order_serial_number = 0 #訂單編號
    before_purchase_counts=[]
    i=0
    
#     session_count_index = [] #儲存session count的初始row index 以及跳動發生的時候 之index
#     session_series_count_index = [] # 儲存session series count跟上述一樣的東西 目前還不確定正不正確
    
    
#    behavior_name =''
    
    purchase_amount=0
    discount_amount=0
    
#     sessiontime = pd.to_datetime(file.values[0][0])
#     session_count_index.append(0)
    for i, row in enumerate(file.values):
        
        if(row[3]!='Purchase' and row[3]!='Offline'):
            a_purchase_segment_row += 1
            try:
                a_purchase_segment_source_type["{}".format(row[4])] = a_purchase_segment_source_type["{}".format(row[4])]+1
            except:
                a_purchase_segment_source_type.update({"{}".format(row[4]) : 1})
            try:
                a_purchase_segment_product_list["{}".format(row[5])] = a_purchase_segment_product_list["{}".format(row[5])]+1 #該品項或商品的count+1
            except:
                a_purchase_segment_product_list.update({"{}".format(row[5]) : 1}) 

            #BehaviorType
            try:
                a_purchase_segment_behavior_type["{}".format(row[3])] = a_purchase_segment_behavior_type["{}".format(row[3])]+1
            except:
                a_purchase_segment_behavior_type.update({"{}".format(row[3]) : 1})

            try:
                a_purchase_segment_traffic_source["{}".format(row[1])] = a_purchase_segment_traffic_source["{}".format(row[1])]+1
            except:
                a_purchase_segment_traffic_source.update({"{}".format(row[1]) : 1})


        #算purchase裡，session的跳動  => 11231為跳三次 
        
            if(row[2]!=file.values[i-1][2]): # i-1為上一行 ,意思即是session跳動 i.e.如果session不等於上一個session的number
#                 #if(row[2] + 1 != file.values[i-1][2] ):
#                 session_count_index.append(i-1)
                
                
#                 if(file.values[i+1][3]!='Offline'):
#                     if(file.values[i+1][3]!='Purchase'):
#                         session_count_index.append(i)
                
                
                if((pd.to_datetime(row[0]) - pd.to_datetime(file.values[i-1][0])).total_seconds()>= 1800 or row[4]!=file.values[i-1][4]):  #if 超過1800秒 or 使用來源(web, app)不同:
                    a_purchase_segment_SessionCount += 1 
                    
#                     elif(row[0].split(" ")[0] !=file.values[i-1][0].split(" ")[0] and row[4]!=file.values[i-1][4]):           #elif 換天flag==True and session數兩者不相同:
#                          a_purchase_segment_SessionCount += 1
                elif(row[0].split(" ")[0] ==file.values[i-1][0].split(" ")[0] and row[4]!=file.values[i-1][4]):            #elif 換天flag==False and session數兩者不相同:
                    a_purchase_segment_SessionCount += 1

            
#                 #算SessionSeriesCount
                if(row[2]!=file.values[i-1][2]+1 ): # 如果sessionnumber不連續        
                    if((pd.to_datetime(row[0]) - pd.to_datetime(file.values[i-1][0])).total_seconds()>= 1800 and row[0].split(" ")[0] ==file.values[i-1][0].split(" ")[0]):
                        a_purchase_segment_SessionSeriesCount += 1
#                         session_series_count_index.append(i-1) # 儲存一個session series最後一個row之index
                
#             else: #如果session不跳動
#                 if(row[2]==file.values[i+1][2] and row[2]==1): # 
#                     session_series_count_index.append(i-1)
            
        #如果該筆資料為purchase，把之前計算之數值放入dict,並且全部重新計算
        elif(row[3]=='Purchase' or row[3]=='Offline'): #就是else，為purchase或offline的情況
#             if(file.values[i-1][3]!='Purchase'):
#                 if(file.values[i-1][3]!='Offline'):
#                     if(i != 0):
#                         session_count_index.append(i-1)
            
            
            

            
            if(a_purchase_segment_SessionCount!=0): #把之前count的東西全部裝在一個list
                #計算session,series total time
#                 for index in range(0,len(session_count_index),2):
#                     print(session_count_index)
#                     print(pd.to_datetime(file.values[session_count_index[index+1]][0]))
#                     print(pd.to_datetime(file.values[session_count_index[index]][0]))
#                     #a_purchase_segment_SessionCount_totaltime += (pd.to_datetime(file.values[index+1][0]) - pd.to_datetime(file.values[index][0])).total_seconds()
#                     #print(a_purchase_segment_SessionCount_totaltime)
                
#                 for index in session_series_count_index[::2]:
#                     a_purchase_segment_SessionSeriesCount_totaltime += (pd.to_datetime(file.values[index+1][0]) - pd.to_datetime(file.values[index][0])).total_seconds()                

                
                before_purchase_counts.append({'segment_session_count' : a_purchase_segment_SessionCount,
                                               'segment_session_series_count' : a_purchase_segment_SessionSeriesCount,
                            'source_type' : a_purchase_segment_source_type,
                             'product_list' : a_purchase_segment_product_list,
                            'behavior_type' : a_purchase_segment_behavior_type,
                            'traffic_source' : a_purchase_segment_traffic_source,
#                             'session_count_time' : a_purchase_segment_SessionCount_totaltime,
#                             'session_series_count_time' : a_purchase_segment_SessionSeriesCount_totaltime,
                            'traffic_count' : a_purchase_segment_row
                                               
                                               
                            })

#             a_purchase_segment_SessionCount_totaltime = 0
#             a_purchase_segment_SessionSeriesCount_totaltime = 0
            a_purchase_segment_source_type = {}
            a_purchase_segment_product_list = {}
            a_purchase_segment_row = 0
            a_purchase_segment_SessionCount = 0
            a_purchase_segment_SessionSeriesCount = 1
            a_purchase_segment_behavior_type ={}
            a_purchase_segment_traffic_source ={}
            a_purchase_segment_row = 0
            
            
            #Purchase單 計算

            if( row[0] != file.values[i-1][0]): #為purchase或offline， 且與上一row時間不同
                if(row[0] != file.values[i+1][0] ): #已知下一筆資料不為同一筆訂單=>只購買一個單獨商品的訂單
                    a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1})
                    a_purchase_order['purchase_amount'] = float(row[5].split(" ")[1])
                    a_purchase_order['discount_amount'] = float(row[5].split(" ")[2])         
                    purchase_orders.append({'purchaseOrder{}{}'.format(row[3],purchase_order_serial_number) : a_purchase_order})
                    purchase_amount=0
                    discount_amount=0
                    a_purchase_order = {}
                    purchase_order_serial_number += 1                    
                    continue
                else: # 已知下一筆資料為同筆訂單
                    try:
                        a_purchase_order["{}".format(row[5]).split(" ")[0]] = a_purchase_segment_product_list["{}".format(row[5]).split(" ")[0]]+1
                    except:
                        a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1})
                    purchase_amount += float(row[5].split(" ")[1])
                    discount_amount += float(row[5].split(" ")[2])
                    a_purchase_order['purchase_amount'] = purchase_amount
                    a_purchase_order['discount_amount'] = discount_amount                    
                        


#                 elif(row[0] != file.values[i+1][0] ): #已知下一筆資料不為同一筆訂單=>只購買一個單獨商品的訂單
#                     a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1})
#                     a_purchase_order['purchase_amount'] = float(row[5].split(" ")[1])
#                     a_purchase_order['discount_amount'] = float(row[5].split(" ")[2])         
#                     purchase_orders.append({'purchaseOrder{}{}'.format(row[3],purchase_order_serial_number) : a_purchase_order})
#                     a_purchase_order = {}
#                     purchase_order_serial_number += 1



            else: #如果是 purchase或offline,且時間與上一row時間相同
                if(row[0] == file.values[i+1][0]): #下一row為同筆訂單
                    try:
                        a_purchase_order["{}".format(row[5]).split(" ")[0]] = a_purchase_segment_product_list["{}".format(row[5]).split(" ")[0]]+1
                    except:
                        a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1})
                    purchase_amount += float(row[5].split(" ")[1])
                    discount_amount += float(row[5].split(" ")[2])
                    a_purchase_order['purchase_amount'] = purchase_amount
                    a_purchase_order['discount_amount'] = discount_amount
                else: #同一訂單最後一個row的情況

                    #算最後一次的金額
                    purchase_amount += float(row[5].split(" ")[1])
                    discount_amount += float(row[5].split(" ")[2])
                    a_purchase_order['purchase_amount'] = purchase_amount
                    a_purchase_order['discount_amount'] = discount_amount

                    #add 商品or種類 count
                    try:
                        a_purchase_order["{}".format(row[5]).split(" ")[0]] = a_purchase_segment_product_list["{}".format(row[5]).split(" ")[0]]+1
                    except:
                        a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1}) 


                    purchase_orders.append({'purchaseOrder{}{}'.format(row[3],purchase_order_serial_number) : a_purchase_order})
                    #歸零
                    a_purchase_order = {}
                    purchase_amount=0
                    discount_amount=0
                    purchase_order_serial_number += 1
                    
                    
                    
#         if(i+2 == len(file.index)):
#             if(file.values[i+1][0]==0): # 表示讀到整個資料表最後一row了
#                 if(row[3]!='Purchase' and row[3]!='Offline'):
#                     session_count_index.append(i)
                
                

            
            
            
    return before_purchase_counts,purchase_orders



In [528]:
a = purchaseBeforeCount(file)
a

([{'segment_session_count': 3,
   'segment_session_series_count': 1,
   'source_type': {'APP': 24},
   'product_list': {'35222': 1,
    '33652': 1,
    '183450': 1,
    '4143210': 1,
    '4678808': 1,
    '4700585': 1,
    '4678939': 2,
    '4735719': 1,
    '4721435': 1,
    '86270': 1,
    '4715275': 1,
    '4648175': 2,
    '4721564': 1,
    '4679507': 1,
    '4678921': 2,
    '4678821': 1,
    '4656011': 1,
    '4700252': 1,
    '4636310': 1,
    '4567709': 1,
    '4246981': 1},
   'behavior_type': {'ViewSalePageCategory': 4, 'ViewSalePage': 20},
   'traffic_source': {'Direct': 24},
   'traffic_count': 24},
  {'segment_session_count': 2,
   'segment_session_series_count': 1,
   'source_type': {'APP': 14},
   'product_list': {'51876': 1,
    '4758407': 3,
    '4814269': 2,
    '4814395': 1,
    '4814425': 2,
    '182295': 1,
    '231141': 1,
    '4835216': 1,
    '4835882': 1,
    '35222': 1},
   'behavior_type': {'ViewSalePageCategory': 4, 'ViewSalePage': 10},
   'traffic_source': 